# Métodos III - sobre minímos y máximos


**Course: "Métodos III - Derivadas en varias dimensiones"**

*Author: Jose A. Hernando*, February 2018

*Particle Physics Deparment. Universidade de Santiago de Compostela, Spain.*

In [1]:
# general imports
#%matplotlib inline
%matplotlib

from collections import namedtuple

# numpy and matplotlib
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as optimize

from mpl_toolkits.mplot3d import Axes3D
# possible styles: ggplot (simplicity), bmh (scientify data), 
matplotlib.style.use('ggplot')

Using matplotlib backend: MacOSX


## Likelihood

**Ejemplo:** Generate an experiment that measured n-values of a quantity that is random distributed accordingly with a normal gaussian (mean zero and sigma one).

In [2]:
n = 10000
xs = stats.norm.rvs(size=n)
fig, axs = plt.subplots(2, 1, figsize=(6, 6))
axs[0].plot(xs, ls='none', marker='o')
axs[0].set_xlabel('i-measurement')
axs[1].set_ylabel('measurement')
axs[1].hist(xs, 100);
print('values ', xs[:4])
axs[1].set_xlabel('value')
axs[1].set_ylabel('frequency')
fig.tight_layout();

values  [ 0.67287699  1.60581395 -0.97424339  0.59120874]


In [3]:
n = 10000
xs = stats.norm.rvs(size=n)

def likelihood(xs, x):
    pis = np.array([stats.norm.pdf(xi, x) for xi in xs])
    return np.prod(pis)

# loglikelihood
def ll(x, xs):
    dx = (x-xs)
    return np.sum(dx*dx)

fig, ax = plt.subplots()

ax.hist(xs, bins=50, alpha=0.5, range=(-3., 3.), color='blue')
axb = ax.twinx()
mus = np.linspace(-3., 3., 100)
lls = np.array([ll(mu, xs) for mu in mus])
axb.plot(mus, lls, alpha=0.5, color='red', lw=2)
ax.set_xlabel('$x$', fontsize=16)
ax.set_ylabel('samples'                        , color='blue', fontsize=14);
axb.set_ylabel('$-2 \, \log \, \mathcal{L}(x)$', color='red' , fontsize=14);

## Least squared fit

In [4]:
FitResult = namedtuple('FitResult', 'params cov chi2')
xsize = 100.

def experiment(m, atrue=1, btrue=0, sigma=1.):
    xdata = xsize*stats.uniform.rvs(size=m)
    ydata = atrue*xdata + btrue + stats.norm.rvs(scale=sigma, size=m)
    return (xdata, ydata)

def straight_line(x, a, b):
    return x*a + b

def fit_experiment(xdata, ydata, func, npars=2):
    popt, pcov = optimize.curve_fit(func, xdata, ydata)
    ypred = func(xdata, *popt)
    chi2 = np.sum((ydata-ypred)*(ydata-ypred)/(sigma*sigma))/(1.*(len(ydata)-len(popt)))
    return FitResult(popt, pcov, chi2)

In [5]:
atrue, btrue, sigma = 1., 0., 1.
m = 10
xdata, ydata = experiment(m, atrue, btrue, sigma)
fitresult    = fit_experiment(xdata, ydata, straight_line)

ahat, bhat = fitresult.params

fig, ax = plt.subplots(figsize=(8, 8))
print('estimated parameters:', 'a = ', ahat, ', b= ',bhat)

# plot data
ax.plot(xdata, ydata             , color='black', marker='o', markersize=5, ls='none', label='data')
ax.plot(xdata,  ahat*xdata + bhat, color='blue' , lw=2, label='ls fit', alpha=0.5)
ax.plot(xdata, atrue*xdata + btrue, color='green', lw=2,  label='true'  , alpha=0.5)
ax.legend(loc=2, fontsize=14)
ax.set_xlabel(r'$x$', fontsize=14)
ax.set_ylabel(r'$y$', fontsize=14);

estimated parameters: a =  1.00953782095 , b=  -0.762303709705


**Example:** Generate n-large experiments with m-samples and obtaine the distribution of the $\chi^2/ndf$, the parameters and the pulls.

In [6]:
n = 10000   # number of experiments
m = 4     # number of points per experiment
atrue, btrue = 1, 0
nbins = 40

datas      = [experiment(m, atrue, btrue) for i in range(n)]
fitresults = [fit_experiment(xdata, ydata, straight_line) for xdata, ydata in datas]

fig, axs = plt.subplots(3, 2, figsize=(9, 12))

# parameters
ahats = [fitresult.params[0] for fitresult in fitresults]
axs[0, 0].hist(ahats, nbins)
axs[0, 0].set_xlabel('$\hat{a}$')
bhats = [fitresult.params[1] for fitresult in fitresults]
axs[0, 1].hist(bhats, nbins)
axs[0, 1].set_xlabel('$\hat{b}$')

# errors
perrors = [np.sqrt(np.diag(fitresult.cov)) for fitresult in fitresults]
asigmas = [perror[0] for perror in perrors]
bsigmas = [perror[1] for perror in perrors]
axs[1, 0].hist(asigmas, nbins)
axs[1, 0].set_xlabel('$\sigma_a$')
axs[1, 1].hist(bsigmas, nbins)
axs[1, 1].set_xlabel('$\sigma_b$')

# pulls
apulls = (np.array(ahats)-atrue)/np.array(asigmas)
bpulls = (np.array(bhats)-btrue)/np.array(bsigmas)
print('a pull: mu = ', np.mean(apulls), 'sigma = ', np.std(apulls))
print('b pull: mu = ', np.mean(bpulls), 'sigma = ', np.std(bpulls))
axs[2, 0].hist(apulls, nbins)
axs[2, 0].set_xlabel('$(\hat{a}-a_{True})/\sigma_a$')
axs[2, 1].hist(bpulls, nbins)
axs[2, 1].set_xlabel('$(\hat{b}-b_{True})/\sigma_b$')

fig.tight_layout()

fig, ax = plt.subplots()
chi2 = [fitresult.chi2 for fitresult in fitresults]
print('chi2: mean ', np.mean(np.array(chi2)))
ax.hist(chi2, nbins)
ax.set_xlabel('$\chi^2/ndf$')
fig.tight_layout()

a pull: mu =  0.00655757694981 sigma =  3.11781209927
b pull: mu =  -0.0146856501754 sigma =  3.10836521071
chi2: mean  0.996337590693


## About NN

## Neural Networks

Neural Networks (NN) is a web of nodes (neurons) organized in layers. Each neuron gets several inputs (the initial values or the output or other neurons) and response with an output usually in the $[0, 1]$ range. The last layer has a neuron that provides the target decision. NN mimics the human nervous system. 

A neuron response is a faction of a linear combination of its $n$-inputs ($x_i$). It has $n+1$ parameters:

$$
u(a_0 + \sum_{i=1}^n a_i \, x_i)
$$

The function $u(x)$ is usually a sigmoid:
$$
u(x) = \frac{1}{1+e^{-x}}
$$

NN are trained via down-back modification of neurons parameters via minimizing an error  function for the last neuron:

$$
E(x, y) = \sum_i (u(x_i)-y_i)^2
$$

where $x_i$ are the samples and $y_i$ the target. 

Recently, new computational techniques, has boosted NN, allowing to have a greather number of layers and a faster learning rate. These NN are called Deep Neural Network, for deep learning. This revolution was started by the Google company.

In [7]:
xs = np.linspace(-1., 1., 100)
w =  100.
b =  0.
ys = 1./(1.+np.exp(-w*xs-b))
plt.plot(xs, ys);

### load the data

In [8]:
%matplotlib

Using matplotlib backend: MacOSX


In [9]:
# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics

# The digits dataset
digits = datasets.load_digits()
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

X_train = data[:n_samples//2]
y_train = digits.target[:n_samples//2]


def plot_digit(index, show=True):
    image, target = digits.images[index], digits.target[index]
    plt.subplot()
    plt.imshow(image, cmap=plt.cm.inferno, interpolation='nearest')
    if (show):
        plt.title('true: %i' % target)

In [10]:
index = 163
show = 0
plot_digit(index, show = show)

In [11]:
from sklearn.neural_network import MLPClassifier

n_train = n_samples // 2 

X_train = data         [        : n_train]
y_train = digits.target[        : n_train]

X_test  = data         [n_train :        ]
y_test  = digits.target[n_train :        ]


classifier = MLPClassifier(solver='adam')

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)

# Now predict the value of the digit on the second half:
y_predicted = classifier.predict(X_test)
#plt.hist(y_predicted, 100);

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.grid(False)
_, _, _, ct = ax.hist2d(y_test, y_predicted, cmap=plt.cm.jet);
ax.set_xlabel('y-test')
ax.set_ylabel('y-predicted');
plt.colorbar(ct, ax=ax);

In [12]:
oks = [yp == yt for yp, yt in zip(y_predicted, y_test)]
#print(oks)
index = 6
show  = 1
if (show):
    print(' Predicted :', y_predicted[index], ', True : ', y_test[index])
plot_digit(n_train+index, show = show)


 Predicted : 9 , True :  9


In [13]:
from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)


# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)


# #############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))


# #############################################################################
# Train a SVM classification model

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)


# #############################################################################
# Quantitative evaluation of the model quality on the test set

print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))


# #############################################################################
# Qualitative evaluation of the predictions using matplotlib

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

2018-03-06 10:37:52,894 Downloading LFW data (~200MB): http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz


Automatically created module for IPython interactive environment


2018-03-06 10:38:52,583 Decompressing the data archive to /Users/hernando/scikit_learn_data/lfw_home/lfw_funneled
2018-03-06 10:39:06,403 Loading LFW people faces from /Users/hernando/scikit_learn_data/lfw_home
2018-03-06 10:39:06,823 Loading face #00001 / 01288
2018-03-06 10:39:11,205 Loading face #01001 / 01288


Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7
Extracting the top 150 eigenfaces from 966 faces
done in 0.480s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.034s
Fitting the classifier to the training set
done in 24.261s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting people's names on the test set
done in 0.054s
                   precision    recall  f1-score   support

     Ariel Sharon       0.57      0.62      0.59        13
     Colin Powell       0.74      0.88      0.80        60
  Donald Rumsfeld       0.73      0.70      0.72        27
    George W Bush       0.90      0.88      0.89       146
Gerhard Schroeder       0.83      0.76      0.79        25
      Hugo Chavez       0.73      0.53      0.62     